## Problem 5
### EM with Binomial Distribution:
Two biased coins mixed together:
- Coin 1: Shows heads with probability p
- Coin 2: Shows heads with probability r
- Mixing weights: pick coin 1 with probability->$$\pi$$
- M : Number of sequences
- K : number of flips per seqeuence

In [23]:
import numpy as np

### Part A - Generate Mix data and EM Algorithm:

In [24]:
def generate_coin_mix_data(pi, p, r, M = 100, K = 10):
    data = []
    true_coins = []     # this keeps track of which coin was used for verification

    for i in range(M):
        if np.random.random() < pi:
            coin = 1
            prob_heads = p
        else:
            coin = 2
            prob_heads = r

        # 1 for heads & 0 for tails
        flips = np.random.binomial(1, prob_heads, K)

        data.append(flips)
        true_coins.append(coin)

    return np.array(data), np.array(true_coins)

In [25]:
# data generation check:
pi = 0.75
p = 0.8
r = 0.4

data, true_coins = generate_coin_mix_data(pi, p, r)
print(f"Data shape: {data.shape}")
print(f"Sample sequences:\n{data[:3]}")
print(f"Number of heads per sequence: {np.sum(data, axis=1)[:10]}")

Data shape: (100, 10)
Sample sequences:
[[1 1 1 1 1 1 1 1 1 1]
 [1 1 1 0 0 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 0]]
Number of heads per sequence: [10  8  9 10  3  8  9  9  8  4]


In [26]:
def EM_Binomial_Mix(data, max_iter=100, tol=1e-6):
    M, K = data.shape

    # initializing params randomly:
    pi_rand = np.random.random()
    p_rand = np.random.random()
    r_rand = np.random.random()

    print(f"Initial: pi={pi_rand:.3f}, p={p_rand:.3f}, r={r_rand:.3f}")

    log_likelihoods = []

    for iteration in range(max_iter):
        """
        E - STEP:
        Calc responsibilities => which coin likely to generate each sequence
        """

        n_heads = np.sum(data, axis=1)
        n_tails = K - n_heads

        """
        Likelihood of each sequence given coin 1
        P(sequence | coin 1) = C(K, n_heads) * p^n_heads * (1-p)^n_tails
        """
        likelihood_coin1 = (p_rand ** n_heads) * ((1-p_rand) ** n_tails)

        likelihood_coin2 = (r_rand ** n_heads) * ((1-r_rand) ** n_tails)

        """
        Joint Probability: P(sequence, coin) = P(sequence | coin) * P(coin)
        """
        joint_coin1 = likelihood_coin1 * pi_rand
        joint_coin2 = likelihood_coin2 * (1-pi_rand)

        """
        Responsibilities: P(coin|sequence) -> with bayes rule
        """
        total = joint_coin1 + joint_coin2 + 1e-10
        gamma_coin1 = joint_coin1 / total
        gamma_coin2 = joint_coin2 / total

        """
        M - STEP:
        Update parameters based on responsibilities
        Update Mixing weight
        """
        pi_rand_new = np.mean(gamma_coin1)

        """
        Update coin probabilities for both coins
        Weighted average of heads frequency
        """
        weighted_sum_coin1 = np.sum(gamma_coin1[:, np.newaxis] * data)
        weighted_total_coin1 = np.sum(gamma_coin1) * K
        p_rand_new = weighted_sum_coin1 / weighted_total_coin1

        weighted_sum_coin2 = np.sum(gamma_coin2[:, np.newaxis] * data)
        weighted_total_coin2 = np.sum(gamma_coin2) * K
        r_rand_new = weighted_sum_coin2 / weighted_total_coin2

        """
        calc log-likelihood and convergence step
        """
        log_likelihood = np.sum(np.log(total))
        log_likelihoods.append(log_likelihood)

        if iteration>0:
            if abs(pi_rand_new - pi_rand) < tol and abs(p_rand_new - p_rand) < tol and abs(r_rand_new - r_rand) < tol:
                print(f"Converged at iteration {iteration}")
                break

        pi_rand = pi_rand_new
        p_rand = p_rand_new
        r_rand = r_rand_new

        if iteration % 10 == 0:
            print(f"Iter {iteration}: pi={pi_rand:.3f}, p={p_rand:.3f}, r={r_rand:.3f}")

    return pi_rand, p_rand, r_rand, log_likelihoods

In [27]:
# Running em on data
pi_recovered, p_recovered, r_recovered, log_likelihoods = EM_Binomial_Mix(data)

print("\n" + "="*50)
print("True parameters:")
print(f"  pi={pi:.3f}, p={p:.3f}, r={r:.3f}")
print("Recovered parameters:")
print(f"  pi={pi_recovered:.3f}, p={p_recovered:.3f}, r={r_recovered:.3f}")

Initial: pi=0.586, p=0.902, r=0.264
Iter 0: pi=0.710, p=0.811, r=0.450
Iter 10: pi=0.698, p=0.803, r=0.483
Iter 20: pi=0.694, p=0.803, r=0.485
Iter 30: pi=0.693, p=0.804, r=0.485
Iter 40: pi=0.693, p=0.804, r=0.485
Iter 50: pi=0.693, p=0.804, r=0.485
Converged at iteration 52

True parameters:
  pi=0.750, p=0.800, r=0.400
Recovered parameters:
  pi=0.693, p=0.804, r=0.485


### Part - B : Testing EM with multiple param sets

In [28]:
import numpy as np
import pandas as pd

def test_EM_multi_param():
    """
    Part B: Test EM algorithm with multiple parameter sets
    Report results in a comparison table
    """
    test_cases = [
        {'pi': 0.75, 'p': 0.8, 'r': 0.4, 'description': 'Well separated'},
        {'pi': 0.5, 'p': 0.7, 'r': 0.3, 'description': 'Equal mixing'},
        {'pi': 0.3, 'p': 0.6, 'r': 0.2, 'description': 'Unequal mixing (pi<0.5)'},
        {'pi': 0.6, 'p': 0.65, 'r': 0.35, 'description': 'Similar probabilities'},
        {'pi': 0.8, 'p': 0.9, 'r': 0.1, 'description': 'Extreme probabilities'},
        {'pi': 0.4, 'p': 0.55, 'r': 0.45, 'description': 'Very similar coins'},
        {'pi': 0.7, 'p': 0.85, 'r': 0.25, 'description': 'Moderate separation'},
        {'pi': 0.25, 'p': 0.5, 'r': 0.1, 'description': 'Low mixing weight'}
    ]

    results_list = []

    print("="*80)
    print("TESTING EM ALGORITHM WITH MULTIPLE PARAMETER SETS")
    print("="*80)

    for idx, params in enumerate(test_cases, 1):
        print(f"\n--- Test Case {idx}: {params['description']} ---")
        print(f"True Parameters: π={params['pi']:.3f}, p={params['p']:.3f}, r={params['r']:.3f}")

        # Generate data with true parameters
        data, _ = generate_coin_mix_data(params['pi'], params['p'], params['r'], M=100, K=10)

        best_result = None
        best_log_likelihood = -np.inf

        for run in range(5):
            pi_rec, p_rec, r_rec, log_liks = EM_Binomial_Mix(data, max_iter=100)

            if len(log_liks) > 0 and log_liks[-1] > best_log_likelihood:
                best_log_likelihood = log_liks[-1]
                best_result = (pi_rec, p_rec, r_rec)

        pi_rec, p_rec, r_rec = best_result

        error1 = abs(params['pi'] - pi_rec) + abs(params['p'] - p_rec) + abs(params['r'] - r_rec)
        error2 = abs(params['pi'] - (1-pi_rec)) + abs(params['p'] - r_rec) + abs(params['r'] - p_rec)

        if error2 < error1:
            pi_rec = 1 - pi_rec
            p_rec, r_rec = r_rec, p_rec
            print("(Label switching detected and corrected)")

        print(f"Recovered:       π={pi_rec:.3f}, p={p_rec:.3f}, r={r_rec:.3f}")

        results_list.append({
            'Test Case': idx,
            'Description': params['description'],
            'True π': params['pi'],
            'Recovered π': round(pi_rec, 4),
            'π Error': round(abs(params['pi'] - pi_rec), 4),
            'True p': params['p'],
            'Recovered p': round(p_rec, 4),
            'p Error': round(abs(params['p'] - p_rec), 4),
            'True r': params['r'],
            'Recovered r': round(r_rec, 4),
            'r Error': round(abs(params['r'] - r_rec), 4),
            'Total Error': round(abs(params['pi'] - pi_rec) +
                               abs(params['p'] - p_rec) +
                               abs(params['r'] - r_rec), 4)
        })

    results_df = pd.DataFrame(results_list)

    print("\n" + "="*80)
    print("COMPARISON TABLE: TRUE VS RECOVERED PARAMETERS")
    print("="*80)
    print(results_df.to_string(index=False))

    print("\n" + "="*80)
    print("SUMMARY STATISTICS")
    print("="*80)
    print(f"Average π error: {results_df['π Error'].mean():.4f}")
    print(f"Average p error: {results_df['p Error'].mean():.4f}")
    print(f"Average r error: {results_df['r Error'].mean():.4f}")
    print(f"Average total error: {results_df['Total Error'].mean():.4f}")

    return results_df

# Run Part B
results = test_EM_multi_param()

TESTING EM ALGORITHM WITH MULTIPLE PARAMETER SETS

--- Test Case 1: Well separated ---
True Parameters: π=0.750, p=0.800, r=0.400
Initial: pi=0.007, p=0.261, r=0.925
Iter 0: pi=0.165, p=0.333, r=0.799
Iter 10: pi=0.191, p=0.375, r=0.804
Iter 20: pi=0.194, p=0.378, r=0.805
Iter 30: pi=0.195, p=0.378, r=0.805
Iter 40: pi=0.195, p=0.378, r=0.805
Converged at iteration 41
Initial: pi=0.839, p=0.199, r=0.131
Iter 0: pi=0.977, p=0.729, r=0.423
Iter 10: pi=0.817, p=0.802, r=0.366
Iter 20: pi=0.806, p=0.805, r=0.377
Iter 30: pi=0.805, p=0.805, r=0.378
Iter 40: pi=0.805, p=0.805, r=0.378
Converged at iteration 46
Initial: pi=0.567, p=0.067, r=0.710
Iter 0: pi=0.086, p=0.236, r=0.768
Iter 10: pi=0.186, p=0.369, r=0.803
Iter 20: pi=0.194, p=0.377, r=0.805
Iter 30: pi=0.195, p=0.378, r=0.805
Iter 40: pi=0.195, p=0.378, r=0.805
Converged at iteration 45
Initial: pi=0.046, p=0.513, r=0.119
Iter 0: pi=0.868, p=0.785, r=0.306
Iter 10: pi=0.811, p=0.804, r=0.372
Iter 20: pi=0.806, p=0.805, r=0.378
Iter

### Part - C : T param